# Example Of Pyspark ML

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Missing").getOrCreate()

In [3]:
training = spark.read.csv("./Pyspark-With-Python/test1.csv", header=True, inferSchema=True)

In [4]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [5]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [6]:
training.columns

['Name', 'age', 'Experience', 'Salary']

[Age, Experience]---->new feature----->independent feature

In [7]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=['age', 'Experience'], outputCol="Independent features")

In [8]:
output = featureassembler.transform(training)

In [9]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [10]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent features']

In [11]:
finalized_data = output.select("Independent features", "Salary")

In [24]:
finalized_data.show()

+--------------------+------+
|Independent features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [26]:
from pyspark.ml.regression import LinearRegression
# train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [27]:
train_data.show()

+--------------------+------+
|Independent features|Salary|
+--------------------+------+
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
|          [29.0,4.0]| 20000|
|          [30.0,8.0]| 25000|
+--------------------+------+



In [28]:
test_data.show()

+--------------------+------+
|Independent features|Salary|
+--------------------+------+
|          [24.0,3.0]| 20000|
|         [31.0,10.0]| 30000|
+--------------------+------+



In [29]:
regressor.coefficients

DenseVector([109.3058, 1199.4092])

In [30]:
regressor.intercept

12187.592319054227

In [31]:
pred_results=regressor.evaluate(test_data)

In [32]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent features|Salary|        prediction|
+--------------------+------+------------------+
|          [24.0,3.0]| 20000| 18409.15805022155|
|         [31.0,10.0]| 30000|27570.162481536143|
+--------------------+------+------------------+



In [33]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(2010.339734121153, 4217444.237654746)